**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [1]:
#v20提交成绩得到了3.688， cv 成绩3.6519
#使用authorized_flag 进行特征分别处理,收到了奇效
#使用countVectorizer 对类别变量进行编码，内存存在压力

In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
warnings.filterwarnings('ignore')
np.random.seed(4950)

In [2]:
import pickle

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv')

In [5]:
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [6]:
for df in [df_hist_trans,df_new_merchant_trans]:
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    #修改特征
    df['category_2'].fillna(-1,inplace=True)
    df['category_3'].fillna('other',inplace=True)
    df['merchant_id'].fillna('other',inplace=True)
#     df.loc[df['installments'].isin([999,-1]),'installments'] = 0 

In [7]:
def get_new_columns(name,aggs):
    #for for 写法 nice
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [8]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime(2018,6,1) - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [9]:
#因为根据auth_flag 将特征分成了两部分，这里聚合一个全局的auth_flag 特征
aggs = {}
aggs['purchase_amount'] = ['sum']
# aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['authorized_flag'] = ['sum', 'mean','std']
aggs['card_id'] = ['size']
auth_flag = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_flag.columns = get_new_columns('auth_flag',aggs)
auth_flag.reset_index(inplace=True)
df_train = df_train.merge(auth_flag,on='card_id',how='left')
df_test = df_test.merge(auth_flag,on='card_id',how='left')
del auth_flag
gc.collect()

69

In [10]:
#对 authorized_flag进行结果编码
aggs = {}
for col in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
    df_hist_trans[col+'_auth_mean'] = df_hist_trans.groupby([col])['authorized_flag'].transform('mean')
    df_hist_trans[col+'_auth_sum'] = df_hist_trans.groupby([col])['authorized_flag'].transform('sum') 
    aggs[col+'_auth_mean'] = ['mean']
    aggs[col+'_auth_sum'] = ['sum'] 
auth_encoder = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_encoder.columns = get_new_columns('auth_encoder',aggs)
auth_encoder.reset_index(inplace=True)
df_train = df_train.merge(auth_encoder,on='card_id',how='left')
df_test = df_test.merge(auth_encoder,on='card_id',how='left')
del auth_encoder
gc.collect()

49

In [11]:
def aggregate_per_month(prefix,history,agg_func):
    grouped = history.groupby(['card_id', 'month_lag'])
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = [prefix + '_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = [prefix + '_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True) 
    return final_group

In [12]:
#对授权码 进行按月聚合
agg_func = {'authorized_flag': [ 'sum', 'mean','median']}
final_group =  aggregate_per_month('agg_per_month_total',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group

In [13]:
authorized_transactions = df_hist_trans[df_hist_trans['authorized_flag'] == 1]
df_hist_trans = df_hist_trans[df_hist_trans['authorized_flag'] == 0]

In [ ]:
#使用countVectorizer对category特征进行处理，别人号称可以提升 千分之3
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(token_pattern='\w{1,}')   

In [14]:
df_hist_trans.shape

(2516909, 33)

In [15]:
agg_func = {'purchase_amount': [ 'sum', 'mean', 'min', 'max', 'std']}
final_group =  aggregate_per_month('agg_per_month_auth',authorized_transactions,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist_new',df_new_merchant_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
gc.collect()

85

In [16]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [17]:
i = 0
for df in [authorized_transactions,df_hist_trans]:
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
                'state_id','city_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
    aggs['installments'] = ['sum','max','min','mean','var','median']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','median']
    aggs['month_diff'] = ['mean','median']
#     aggs['authorized_flag'] = ['sum', 'mean','median']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    #产生交叉特征，内存有问题
    features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#     for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#             df[coli + colj] = df[coli].astype(str) + df[colj].astype(str)
#             features.append(coli + colj)
    for col in features:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum') 
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum'] 
#         添加特征，使用outlier进行编码
#         outliers_mean = df.groupby([col])['outliers'].mean()
#         outliers_sum = df.groupby([col])['outliers'].sum()
#         df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#         df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#         aggs[col+'_outliers_mean'] = ['mean']
#         aggs[col+'_outliers_sum'] =['sum']   
        
    if i == 0:
        prefix = 'auth_hist'
    else:
        prefix = 'hist'
    new_columns = get_new_columns(prefix,aggs)
    i += 1
    # df_hist_trans.sort_values(['card_id','purchase_date'],inplace = True)

    df_hist_trans_group = df.groupby('card_id').agg(aggs)
    df_hist_trans_group.columns = new_columns
    df_hist_trans_group.reset_index(drop=False,inplace=True)
    df_hist_trans_group[prefix + '_purchase_date_diff'] = (df_hist_trans_group[prefix + '_purchase_date_max'] - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    df_hist_trans_group[prefix + '_purchase_date_average'] = df_hist_trans_group[prefix + '_purchase_date_diff']/df_hist_trans_group[prefix + '_card_id_size']
    df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (datetime.datetime(2018,6,1) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    #下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
#     df_hist_trans_group[prefix + '_purchase_date_uptonow_ave'] =  df_hist_trans_group[prefix + '_purchase_date_uptonow']/df_hist_trans_group[prefix + '_purchase_date_average']

    #每一个card中未授权消费次数
#     df_hist_trans_group[prefix + '_unauthorized_number'] = df_hist_trans_group[prefix + '_card_id_size'] - df_hist_trans_group[prefix + '_authorized_flag_sum']
    #最近活跃时间，确实是一个强特征，感觉可以再挖出来几个特征，比如最近5次消费时间，最近10次消费时间，如果值比较小，说明最近很活跃
    #没有效果
    # grouped =  df_hist_trans.groupby('card_id')['purchase_date']
    # df_hist_trans_group['hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(5)).dt.days
    # df_hist_trans_group['hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(3)).dt.days
    # df_hist_trans_group['hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(10)).dt.days
    df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
    df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
    del df_hist_trans_group
    time.sleep(5)
    gc.collect()
    del df;gc.collect()
    time.sleep(5)

In [18]:
aggs = {}
#添加特征
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
            'state_id','city_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','median']
aggs['month_diff'] = ['mean','median']
aggs['weekend'] = ['sum', 'mean']
aggs['authorized_flag'] = ['sum', 'mean','median']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

#添加特征   
features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#产生交叉特征，内存有问题
# for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#     for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         df_new_merchant_trans[coli + colj] = df_new_merchant_trans[coli].astype(str) + df_new_merchant_trans[colj].astype(str)
#         features.append(coli + colj)
for col in features:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    df_new_merchant_trans[col+'_sum'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']
    aggs[col+'_sum'] = ['sum']
    #添加特征，使用outlier进行编码
#     outliers_mean = df.groupby([col])['outliers'].mean()
#     outliers_sum = df.groupby([col])['outliers'].sum()
#     df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#     df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#     aggs[col+'_outliers_mean'] = ['mean']
#     aggs[col+'_outliers_sum'] =['sum']   
    
new_columns = get_new_columns('new_hist',aggs)
# df_new_merchant_trans.sort_values(['card_id','purchase_date'],inplace = True)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime(2018,6,1) - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
#下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
# df_hist_trans_group['new_hist_purchase_date_uptonow_ave'] =  df_hist_trans_group['new_hist_purchase_date_uptonow']/df_hist_trans_group['new_hist_purchase_date_average']

#每一个card中未授权消费次数
df_hist_trans_group['new_hist_unauthorized_number'] = df_hist_trans_group['new_hist_card_id_size'] - df_hist_trans_group['new_hist_authorized_flag_sum']
# grouped = df_new_merchant_trans.groupby('card_id')['purchase_date']
# df_hist_trans_group['new_hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(5)).dt.days
# df_hist_trans_group['new_hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(3)).dt.days
# df_hist_trans_group['new_hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(10)).dt.days

df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()
del df_new_merchant_trans
gc.collect()
time.sleep(5)

In [19]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime(2018,6,1) - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #添加特征
    df['auth_hist_first_buy'] = (df['auth_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #修改特征
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min','auth_hist_purchase_date_max','auth_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    for f in ['auth_encoder_category_2_auth_sum_sum','auth_encoder_category_3_auth_sum_sum',
            'auth_encoder_state_id_auth_sum_sum','auth_encoder_subsector_id_auth_sum_sum',
            'auth_encoder_merchant_category_id_auth_sum_sum','auth_encoder_city_id_auth_sum_sum']:
        df[f] = df[f].astype(np.int64) * 1e-9
    #上面auth_flag已经聚合过了card_id_size ,purchase_amount
#     df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']  + df['auth_hist_card_id_size']
#     df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']+df['auth_hist_purchase_amount_sum']
#添加特征

for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    order_label1 = df_train.groupby([f])['outliers'].mean()
    df_train[f+'_outliers_mean'] = df_train[f].map(order_label1)
    df_test[f+'_outliers_mean'] = df_test[f].map(order_label1)
    
    order_label2 = df_train.groupby([f])['outliers'].sum()
    df_train[f+'_outliers_sum'] = df_train[f].map(order_label2)
    df_test[f+'_outliers_sum'] = df_test[f].map(order_label2)
    
#     order_label1 = df_train.groupby([f])['target'].mean()
#     df_train[f+'_target_mean'] = df_train[f].map(order_label1)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label1)
#     order_label2 = df_train.gorupby([f])['target'].sum()
#     df_train[f+'_target_sum'] = df_train[f].map(order_label2)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label2)
 
# get_dummies 似乎有一点点不良影响
df_train = pd.get_dummies(df_train,columns =['feature_1','feature_2'])
df_test = pd.get_dummies(df_test,columns =['feature_1','feature_2'])

In [20]:
#首次购买的时间居然早于首次激活的时间，进行调整
df_train.loc[df_train['auth_hist_first_buy'] < 0, 'auth_hist_first_buy'] = -1
df_train.loc[df_train['hist_first_buy'] < 0, 'hist_first_buy'] = -1
df_train.head()

,first_active_month,card_id,feature_3,target,auth_flag_purchase_amount_sum,auth_flag_authorized_flag_sum,auth_flag_authorized_flag_mean,auth_flag_authorized_flag_std,auth_flag_card_id_size,auth_encoder_category_2_auth_mean_mean,...,dayofweek_outliers_mean,dayofweek_outliers_sum,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3
0,2017-06-01,C_ID_92a2005557,1,-0.820283,-165.968735,247,0.950000,0.218365,260,0.927583,...,0.015174,401,0,0,0,0,1,0,1,0
1,2017-01-01,C_ID_3d0044924f,0,0.392913,-210.006332,339,0.968571,0.174723,350,0.914943,...,0.009631,289,0,0,0,1,0,1,0,0
2,2016-08-01,C_ID_d639edf6cd,0,0.688056,-29.167391,41,0.953488,0.213083,43,0.933632,...,0.013597,247,0,1,0,0,0,0,1,0
3,2017-09-01,C_ID_186d6a6901,0,0.142495,-49.491364,77,1.000000,0.000000,77,0.905491,...,0.005835,201,0,0,0,1,0,0,0,1
4,2017-11-01,C_ID_cdbd2c0db2,0,-0.159749,-48.687656,128,0.962406,0.190931,133,0.911664,...,0.011268,409,1,0,0,0,0,0,0,1


In [21]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features ]
target = df_train['target']
del df_train['target']

In [22]:
len(df_train_columns)
len(exclude_features)

4

## Above is the same with wang, I call it wang's features

### I replace wang's stratified CV with normal lgb.cv

In [35]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "random_state": 4950}

In [36]:
trn_data = lgb.Dataset(df_train[df_train_columns], label=target)#, categorical_feature=categorical_feats)

In [37]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's rmse: 3.68615 + 0.0282081
[400]	cv_agg's rmse: 3.6668 + 0.0265823
[600]	cv_agg's rmse: 3.65987 + 0.0255071
[800]	cv_agg's rmse: 3.6573 + 0.0250177
[1000]	cv_agg's rmse: 3.65601 + 0.0250614
[1200]	cv_agg's rmse: 3.65523 + 0.0246955
[1400]	cv_agg's rmse: 3.65529 + 0.024449
[1600]	cv_agg's rmse: 3.65535 + 0.024517
[1800]	cv_agg's rmse: 3.65527 + 0.0247631


lgb.cv show's that num_boost_round is around 2000, so we train a model with num_boost_round=2000

In [38]:
clf = lgb.train(param, trn_data, 2000, valid_sets=(trn_data), verbose_eval=200)

[200]	training's rmse: 3.57876
[400]	training's rmse: 3.49202
[600]	training's rmse: 3.43501
[800]	training's rmse: 3.39164
[1000]	training's rmse: 3.35386
[1200]	training's rmse: 3.3212
[1400]	training's rmse: 3.28996
[1600]	training's rmse: 3.26061
[1800]	training's rmse: 3.23226
[2000]	training's rmse: 3.20493


predict and save

In [41]:
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv.csv', index=False)

## Outlier detection using wang's features

In [14]:
outlier = np.zeros(len(target), dtype=np.int)

In [15]:
outlier[target < -33] = 1

the param is same as regression and just adjust to binary classification

In [20]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "scale_pos_weight": 15,
         "random_state": 4950}

In [22]:
trn_data = lgb.Dataset(train[df_train_columns], label=outlier)#, categorical_feature=categorical_feats)

In [23]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

[200]	cv_agg's auc: 0.904527 + 0.00536859
[400]	cv_agg's auc: 0.904872 + 0.00550933
[600]	cv_agg's auc: 0.904333 + 0.00563883
[800]	cv_agg's auc: 0.903508 + 0.00588203


In [26]:
lgb_cv['auc-mean'][-1]

0.9050613313442047

best num_boost_round is 500, so we use 500 to train a classifer

In [28]:
clf = lgb.train(param, trn_data, 500, verbose_eval=100, valid_sets=(trn_data))

[100]	training's auc: 0.95442
[200]	training's auc: 0.961968
[300]	training's auc: 0.967783
[400]	training's auc: 0.972515
[500]	training's auc: 0.976593


In [29]:
te_prob = clf.predict(test[df_train_columns])

In [30]:
te_prob.min(), te_prob.max()

(0.0004428745997142606, 0.8690567940374151)

In [31]:
wang_out = pd.read_csv('wang_lgb_cv.csv.gz')

In [32]:
wang_out.head()

,card_id,target
0,C_ID_0ab67a22ab,-2.764160
1,C_ID_130fd0cbdd,-0.419947
2,C_ID_b709037bc5,-1.318852
3,C_ID_d27d835a9f,-0.109764
4,C_ID_2b5e3df5c2,-1.263915


In [33]:
wang_out['outlier_prob1'] = te_prob

**outlier_prob2 is just for compare you can ignore**

In [35]:
te_prob_ashish = np.loadtxt('../_ashish3_data/outlier_prob.txt.gz')

In [37]:
wang_out['outlier_prob2'] = te_prob_ashish

In [38]:
wang_out.head()

,card_id,target,outlier_prob1,outlier_prob2
0,C_ID_0ab67a22ab,-2.764160,0.406350,0.574925
1,C_ID_130fd0cbdd,-0.419947,0.011391,0.014464
2,C_ID_b709037bc5,-1.318852,0.088588,0.088909
3,C_ID_d27d835a9f,-0.109764,0.000747,0.001070
4,C_ID_2b5e3df5c2,-1.263915,0.015534,0.009293


In [40]:
wang_out.sort_values('outlier_prob1', ascending=False)

,card_id,target,outlier_prob1,outlier_prob2
78078,C_ID_922f9c5ea6,-22.745534,0.869057,0.862953
88754,C_ID_02871a2207,-15.954320,0.861804,0.854475
118268,C_ID_3420e285b9,-15.433602,0.861054,0.860737
115682,C_ID_099d9774e2,-12.377131,0.855161,0.832710
1772,C_ID_4299911620,-18.860423,0.853740,0.843684
100556,C_ID_70c457436a,-21.097380,0.852554,0.837081
20556,C_ID_aae50409e7,-21.037031,0.852145,0.844683
5008,C_ID_be92f84f5c,-16.025497,0.852141,0.860169
87789,C_ID_bdfdcbb1a0,-19.461891,0.848176,0.832338
17778,C_ID_866eae8fb0,-12.374922,0.848054,0.818394


In [48]:
top20 = _40.card_id.values[:20].tolist()

In [51]:
top20_50 = _40.card_id.values[20:50].tolist()

In [57]:
top20_50_df = wang_out[wang_out.card_id.isin(top20_50)]

In [59]:
top20_50_target = top20_50_df.outlier_prob1 * (-33.21928095)

In [62]:
wang_out.loc[wang_out.card_id.isin(top20_50), 'target'] = top20_50_target

In [63]:
wang_out.loc[wang_out.card_id.isin(top20_50), 'target'] 

4191     -27.843826
5656     -27.955206
16074    -27.933424
17720    -27.995155
19114    -27.624854
21724    -27.652138
23022    -27.753802
26864    -27.703031
29142    -27.745715
32446    -27.730496
33759    -27.713778
40613    -27.710207
43215    -27.871128
44180    -27.787061
44743    -27.694937
54900    -27.716925
58516    -28.019674
63477    -27.659973
68143    -27.997839
72221    -27.992593
75740    -27.780281
102469   -27.614838
104274   -27.762162
104991   -27.854166
108111   -27.779117
117442   -27.634532
119449   -27.689908
120151   -27.980374
120513   -27.633225
122519   -27.665491
Name: target, dtype: float64

In [65]:
wang_out[['card_id', 'target']].to_csv('wang_top20_top50.csv', index=False)

In [2]:
import pandas as pd

In [3]:
test=pd.read_csv('wang_top20_top50.csv')
test.head()

,card_id,target
0,C_ID_0ab67a22ab,-2.243668
1,C_ID_130fd0cbdd,-0.514302
2,C_ID_b709037bc5,-1.196668
3,C_ID_d27d835a9f,-0.112978
4,C_ID_2b5e3df5c2,-1.340485
